In [1]:
import pandas as pd

In [4]:
repo_dir = """C:\\Users\\yjpar\\Documents\\GitHub\\RGTM\\Cleaned\\"""
df1 = pd.read_csv(repo_dir + "merged_manhattan_cleanup.csv")

C:\Users\yjpar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (15,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df1["StreetAdd"] = df1.StreetNum + " " + df1.StreetName.apply(lambda x : x.strip())

In [10]:
df1.groupby(["StreetAdd"])["SalePrice"].count().to_clipboard()

C:\Users\yjpar\Anaconda3\lib\site-packages\pandas\io\clipboards.py:127: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  obj.to_csv(buf, sep=sep, encoding='utf-8', **kwargs)


In [11]:
len(df1.StreetAdd.unique())

11585